In [14]:
# Import all necessary libraries/packages to ingest data from API

import numpy as np
import pandas as pd
import json
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [15]:
ffd_players_API = 'https://www.fantasyfootballdatapros.com/api/players/2019/all'

In [16]:
# The requests library is the de facto standard for making HTTP requests in Python

response = requests.get(ffd_players_API)
if response.status_code == 200:
    print(response)

<Response [200]>


In [17]:
r = requests.get(ffd_players_API)
files = r.json()
print(files)

[{'fumbles_lost': 0.0, 'games_played': 16.0, 'player_name': 'Christian McCaffrey', 'position': 'RB', 'stats': {'passing': {'int': 0.0, 'passing_att': 2.0, 'passing_cmp': 0.0, 'passing_td': 0.0, 'passing_yds': 0.0}, 'receiving': {'receiving_td': 4.0, 'receiving_yds': 1005.0, 'receptions': 116.0, 'targets': 142.0}, 'rushing': {'rushing_att': 287.0, 'rushing_td': 15.0, 'rushing_yds': 1387.0}}, 'team': 'CAR'}, {'fumbles_lost': 2.0, 'games_played': 15.0, 'player_name': 'Lamar Jackson', 'position': 'QB', 'stats': {'passing': {'int': 6.0, 'passing_att': 401.0, 'passing_cmp': 265.0, 'passing_td': 36.0, 'passing_yds': 3127.0}, 'receiving': {'receiving_td': 0.0, 'receiving_yds': 0.0, 'receptions': 0.0, 'targets': 0.0}, 'rushing': {'rushing_att': 176.0, 'rushing_td': 7.0, 'rushing_yds': 1206.0}}, 'team': 'BAL'}, {'fumbles_lost': 3.0, 'games_played': 15.0, 'player_name': 'Derrick Henry', 'position': 'RB', 'stats': {'passing': {'int': 0.0, 'passing_att': 0.0, 'passing_cmp': 0.0, 'passing_td': 0.0, 

In [18]:
# Turn json into actual dataframe format

ffd_players_API_df = pd.json_normalize(files, max_level=1)

In [19]:
ffd_players_API_df.head()

,fumbles_lost,games_played,player_name,position,team,stats.passing,stats.receiving,stats.rushing
0,0.0,16.0,Christian McCaffrey,RB,CAR,"{'int': 0.0, 'passing_att': 2.0, 'passing_cmp'...","{'receiving_td': 4.0, 'receiving_yds': 1005.0,...","{'rushing_att': 287.0, 'rushing_td': 15.0, 'ru..."
1,2.0,15.0,Lamar Jackson,QB,BAL,"{'int': 6.0, 'passing_att': 401.0, 'passing_cm...","{'receiving_td': 0.0, 'receiving_yds': 0.0, 'r...","{'rushing_att': 176.0, 'rushing_td': 7.0, 'rus..."
2,3.0,15.0,Derrick Henry,RB,TEN,"{'int': 0.0, 'passing_att': 0.0, 'passing_cmp'...","{'receiving_td': 2.0, 'receiving_yds': 206.0, ...","{'rushing_att': 303.0, 'rushing_td': 16.0, 'ru..."
3,2.0,16.0,Aaron Jones,RB,GNB,"{'int': 0.0, 'passing_att': 0.0, 'passing_cmp'...","{'receiving_td': 3.0, 'receiving_yds': 474.0, ...","{'rushing_att': 236.0, 'rushing_td': 16.0, 'ru..."
4,2.0,16.0,Ezekiel Elliott,RB,DAL,"{'int': 0.0, 'passing_att': 0.0, 'passing_cmp'...","{'receiving_td': 2.0, 'receiving_yds': 420.0, ...","{'rushing_att': 301.0, 'rushing_td': 12.0, 'ru..."


In [27]:
# Removing nested values from stats.passing, stats.receiving and stats.rushing
# We have a dataframe that has 3 of the columns as a dictionary. 
# We want to unpack it into multiple columns by using apply(pd.Series)


df_passing = ffd_players_API_df['stats.passing'].apply(pd.Series)
df_passing

,int,passing_att,passing_cmp,passing_td,passing_yds
0,0.0,2.0,0.0,0.0,0.0
1,6.0,401.0,265.0,36.0,3127.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
615,0.0,0.0,0.0,0.0,0.0
616,0.0,0.0,0.0,0.0,0.0
617,1.0,4.0,2.0,0.0,14.0
618,0.0,0.0,0.0,0.0,0.0


In [28]:
df_receiving = ffd_players_API_df['stats.receiving'].apply(pd.Series)
df_receiving

,receiving_td,receiving_yds,receptions,targets
0,4.0,1005.0,116.0,142.0
1,0.0,0.0,0.0,0.0
2,2.0,206.0,18.0,24.0
3,3.0,474.0,49.0,68.0
4,2.0,420.0,54.0,71.0
...,...,...,...,...
615,0.0,0.0,0.0,0.0
616,0.0,1.0,1.0,2.0
617,0.0,0.0,0.0,0.0
618,0.0,15.0,2.0,3.0


In [29]:
df_rushing = ffd_players_API_df['stats.rushing'].apply(pd.Series)
df_rushing

,rushing_att,rushing_td,rushing_yds
0,287.0,15.0,1387.0
1,176.0,7.0,1206.0
2,303.0,16.0,1540.0
3,236.0,16.0,1084.0
4,301.0,12.0,1357.0
...,...,...,...
615,0.0,0.0,0.0
616,0.0,0.0,0.0
617,2.0,0.0,-2.0
618,0.0,0.0,0.0


In [33]:
# After removing nested values we want to combine all columns - 
# First we drop old column values from stats.passing, stats.receiving and stats.rushing
# We replace it with our new df_passing, df_receiving, df_rushing

ff_combined_df = pd.concat([ffd_players_API_df.drop(['stats.passing', 'stats.receiving', 'stats.rushing'], axis=1), 
                            df_passing, df_receiving, df_rushing], axis=1)
ff_combined_df

,fumbles_lost,games_played,player_name,position,team,int,passing_att,passing_cmp,passing_td,passing_yds,receiving_td,receiving_yds,receptions,targets,rushing_att,rushing_td,rushing_yds
0,0.0,16.0,Christian McCaffrey,RB,CAR,0.0,2.0,0.0,0.0,0.0,4.0,1005.0,116.0,142.0,287.0,15.0,1387.0
1,2.0,15.0,Lamar Jackson,QB,BAL,6.0,401.0,265.0,36.0,3127.0,0.0,0.0,0.0,0.0,176.0,7.0,1206.0
2,3.0,15.0,Derrick Henry,RB,TEN,0.0,0.0,0.0,0.0,0.0,2.0,206.0,18.0,24.0,303.0,16.0,1540.0
3,2.0,16.0,Aaron Jones,RB,GNB,0.0,0.0,0.0,0.0,0.0,3.0,474.0,49.0,68.0,236.0,16.0,1084.0
4,2.0,16.0,Ezekiel Elliott,RB,DAL,0.0,0.0,0.0,0.0,0.0,2.0,420.0,54.0,71.0,301.0,12.0,1357.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,1.0,6.0,Ray-Ray McCloud,0,CAR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
616,1.0,6.0,Darrius Shepherd,WR,GNB,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0
617,0.0,3.0,Jarrett Stidham,QB,NWE,1.0,4.0,2.0,0.0,14.0,0.0,0.0,0.0,0.0,2.0,0.0,-2.0
618,2.0,7.0,Michael Walker,WR,JAX,0.0,0.0,0.0,0.0,0.0,0.0,15.0,2.0,3.0,0.0,0.0,0.0


In [35]:
ff_combined_df.head(10)

,fumbles_lost,games_played,player_name,position,team,int,passing_att,passing_cmp,passing_td,passing_yds,receiving_td,receiving_yds,receptions,targets,rushing_att,rushing_td,rushing_yds
0,0.0,16.0,Christian McCaffrey,RB,CAR,0.0,2.0,0.0,0.0,0.0,4.0,1005.0,116.0,142.0,287.0,15.0,1387.0
1,2.0,15.0,Lamar Jackson,QB,BAL,6.0,401.0,265.0,36.0,3127.0,0.0,0.0,0.0,0.0,176.0,7.0,1206.0
2,3.0,15.0,Derrick Henry,RB,TEN,0.0,0.0,0.0,0.0,0.0,2.0,206.0,18.0,24.0,303.0,16.0,1540.0
3,2.0,16.0,Aaron Jones,RB,GNB,0.0,0.0,0.0,0.0,0.0,3.0,474.0,49.0,68.0,236.0,16.0,1084.0
4,2.0,16.0,Ezekiel Elliott,RB,DAL,0.0,0.0,0.0,0.0,0.0,2.0,420.0,54.0,71.0,301.0,12.0,1357.0
5,2.0,14.0,Dalvin Cook,RB,MIN,0.0,0.0,0.0,0.0,0.0,0.0,519.0,53.0,63.0,250.0,13.0,1135.0
6,0.0,16.0,Michael Thomas,WR,NOR,0.0,0.0,0.0,0.0,0.0,9.0,1725.0,149.0,185.0,1.0,0.0,-9.0
7,1.0,16.0,Travis Kelce,TE,KAN,0.0,0.0,0.0,0.0,0.0,5.0,1229.0,97.0,136.0,1.0,1.0,4.0
8,3.0,16.0,Nick Chubb,RB,CLE,0.0,0.0,0.0,0.0,0.0,0.0,278.0,36.0,49.0,298.0,8.0,1494.0
9,2.0,16.0,Austin Ekeler,RB,LAC,0.0,0.0,0.0,0.0,0.0,8.0,993.0,92.0,108.0,132.0,3.0,557.0
